In [1]:
import json
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
file_name = "../data/transposition_data/32014L0041-Bulgaria.json"

transpositions = json.load(open(file_name, "r"))
transpositions["items"]

[{'id': 'chap_1',
  'fullId': '',
  'type': 3,
  'text': '<a class="doc-anchor" eid="chap_1"></a><div class="d-EURLEX d-hierarchy d-chapter"></div>',
  'heading': 'CHAPTER I',
  'subParagraph': '',
  'subHeading': 'THE EUROPEAN INVESTIGATION ORDER',
  'comment': None,
  'hasRecitals': False,
  'treeLevel': 0,
  'recitals': [],
  'articles': [],
  'oldArticles': [],
  'hasTransposition': None,
  'transp': None,
  'offence': None,
  'nationalActs': [],
  'regulationCompliance': None,
  'canTranspose': None},
 {'id': 'art_1',
  'fullId': 'chap_1__art_1',
  'type': 5,
  'text': '<a class="doc-anchor" eid="chap_1__art_1"></a><div class="d-EURLEX d-hierarchy d-article"><div class="d-EURLEX d-blocksreq d-content"></div></div>',
  'heading': 'Article 1',
  'subParagraph': '',
  'subHeading': 'The European Investigation Order and obligation to execute it',
  'comment': None,
  'hasRecitals': False,
  'treeLevel': 1,
  'recitals': [],
  'articles': [],
  'oldArticles': [],
  'hasTransposition': 

In [3]:
import re

def remove_html(text):
    html_regex = re.compile(r"<.*?>")
    return re.sub(html_regex, "", text)

Columns of interest: text, heading, subParagraph, hasTransposition, transp, nationalActs->[provisions, title, titleEn]

In [4]:
transpositions["items"][5].keys()
transpositions["items"][5]

{'id': 'art_1__par_3',
 'fullId': 'chap_1__art_1__par_3',
 'type': 6,
 'text': '<p class="d-EURLEX d-inline d-p">3. The issuing of an EIO may be requested by a suspected or accused person, or by a lawyer on his behalf, within the framework of applicable defence rights in conformity with national criminal procedure.</p>',
 'heading': '',
 'subParagraph': '',
 'subHeading': '',
 'comment': '<h3><br></h3>',
 'hasRecitals': False,
 'treeLevel': 2,
 'recitals': [],
 'articles': [],
 'oldArticles': [],
 'hasTransposition': True,
 'transp': {'label': 'Explicitly transposed', 'code': '1'},
 'offence': None,
 'nationalActs': [{'isOfficialTranslation': {'label': 'Official', 'code': '1'},
   'provisions': [{'link': {'docType': 0,
      'itemsBase': [{'label': 'Article', 'value': '5'},
       {'label': 'Paragraph', 'value': ''},
       {'label': 'Alinea', 'value': '2'},
       {'label': 'Point', 'value': ''},
       {'label': 'Letter', 'value': ''},
       {'label': 'Indent', 'value': ''},
       

In [5]:
from collections import defaultdict

def remove_long_label_element(label):
    return "_".join([l for l in label.split("_") if len(l) < 10])

articles = defaultdict(list)

previous_tree_level = -1
nested_texts = []
t = []
for transposition in transpositions["items"]:
    if transposition["treeLevel"] < 0: # skip over chapter/article headings
        continue

    if remove_html(transposition["text"]) == "":
        continue

    articles[transposition["id"].split("__")[0]].append(transposition)

In [ ]:
data = {"eu_id": [], "eu_text": [], "nat_id": [], "nat_text": []}

ids = []
previous_tree_level = -1
nested_texts = []
texts = []

def add_at_level(x, item, current_level, prev_level):
    # print(current_level, prev_level)
    # print(item)
    if len(x) < 1:
        x += [item]
    elif current_level < 1 or current_level == prev_level:
        if type(x) != list:
            x = [x] + item
        else:
            x += item
        return x
    else:
        new_aux = add_at_level(x[-1], item, current_level-1, prev_level)
        x[-1] = new_aux

    return x

combined_items = False
for transposition in transpositions["items"][:20]:
    if remove_html(transposition["text"]) == "":
        continue

    if transposition["treeLevel"] < 1: # skip over chapter/article headings
        continue

    if transposition["treeLevel"] == 1:
        previous_tree_level = -1

    if previous_tree_level < transposition["treeLevel"]:
        print(nested_texts)
        texts.append(nested_texts) if len(nested_texts) > 0 else 1
        print(texts)
        nested_texts = []
        combined_items = False
    
    elif previous_tree_level > transposition["treeLevel"]:
        add_at_level(texts, nested_texts, transposition["treeLevel"], previous_tree_level)
        texts[-1] += [remove_html(transposition["text"])]
        nested_texts = []
        combined_items = True
        
    previous_tree_level = transposition["treeLevel"]
    if not combined_items:
        nested_texts.append(remove_html(transposition["text"]))

    print("-----")

texts

In [338]:
data = {"eu_id": [], "eu_text": [], "nat_id": [], "nat_text": []}

previous_tree_level = -1
nested_texts = []
texts = []

idx = 0

def add_at_level(x, item, current_level, prev_level):
    if len(x) < 1:
        x += [item]
    elif current_level < 1 or current_level == prev_level:
        if type(x) != list:
            x = [x] + item
        else:
            x += item
        return x
    else:
        new_aux = add_at_level(x[-1], item, current_level-1, prev_level)
        x[-1] = new_aux

    return x

while idx < len(transpositions["items"]):
    # print(idx)

    nested_texts = []
    previous_tree_level = transpositions["items"][idx]["treeLevel"]
    orig_tree_level = transpositions["items"][idx]["treeLevel"]
    while idx < len(transpositions["items"]) and transpositions["items"][idx]["treeLevel"] == previous_tree_level:
        text = remove_html(transpositions["items"][idx]["text"])
        nested_texts.append(text) if text != "" else 1
        previous_tree_level = transpositions["items"][idx]["treeLevel"]
        idx += 1

    # print(transpositions["items"][idx]["treeLevel"], previous_tree_level, orig_tree_level)
    # print(nested_texts)
    # print(remove_html(transpositions["items"][idx]["text"]))
    # print(texts)
    # print("------")
    if len(nested_texts) > 0:
        if transpositions["items"][idx]["treeLevel"] > previous_tree_level and len(texts) > 0 and (orig_tree_level != 1):
            if type(texts[-1]) == str:
                texts[-1] = [texts[-1]] + nested_texts
            else:
                texts[-1] = texts[-1] + nested_texts
        elif previous_tree_level > transpositions["items"][idx]["treeLevel"]:
            add_at_level(texts, nested_texts, transpositions["items"][idx]["treeLevel"], previous_tree_level)
        else:
            texts.append(nested_texts) if len(nested_texts) > 0 else 1
    # idx -= 1
    # idx += 1

texts

[['1. A European Investigation Order (EIO) is a judicial decision which has been issued or validated by a judicial authority of a Member State(‘the issuing State’) to have one or several specific investigative measure(s) carried out in another Member State (‘the executing State’) to obtain evidence in accordance with this Directive.',
  'The EIO may also be issued for obtaining evidence that is already in the possession of the competent authorities of the executing State.',
  '2. Member States shall execute an EIO on the basis of the principle of mutual recognition and in accordance with this Directive.',
  '3. The issuing of an EIO may be requested by a suspected or accused person, or by a lawyer on his behalf, within the framework of applicable defence rights in conformity with national criminal procedure.',
  '4. This Directive shall not have the effect of modifying the obligation to respect the fundamental rights and legal principles as enshrined in Article 6 of the TEU, including 

In [342]:
texts[3]

'2. Each Member State shall indicate the language(s) which, among the official languages of the institutions of the Union and in addition to the official language(s) of the Member State concerned, may be used for completing or translating the EIO when the Member State concerned is the executing State.'

In [7]:
data = {"eu_id": [], "eu_text": [], "nat_id": [], "nat_text": []}

ids = []
previous_tree_level = -1
nested_texts = []
t = []
for transposition in transpositions["items"][:15]:
    if transposition["treeLevel"] != previous_tree_level:
        if len(t) > 0 and transposition["treeLevel"] < previous_tree_level:
            if transposition["treeLevel"] == 1:
                print(t)
                print(nested_texts)
                t[-1] = t[-1] + nested_texts
            else:
                # print(remove_html(transposition["text"]))
                t[-1][-1] = [t[-1][-1]] + nested_texts
                # t.append(nested_texts)

        # else:
        #     t[-1] = t[-1] + nested_texts
        t.append(nested_texts) if len(nested_texts) > 0 else 1

        # print(t) if len(t) > 0 else 1
        nested_texts = []
        
    if transposition["treeLevel"] < 0: # skip over chapter/article headings
        continue

    previous_tree_level = transposition["treeLevel"]

    if remove_html(transposition["text"]) == "":
        continue

    if not transposition["hasTransposition"]:
        id = remove_long_label_element(transposition["id"])
        ids.append(id)
        nested_texts.append(remove_html(transposition["text"]))
        # data["eu_text"] += [remove_html(transposition["text"])]
        # data["eu_id"].append(id)
        # data["nat_text"].append("")
        # data["nat_id"].append("")
        continue

    for national_implementation in transposition["nationalActs"]:
        provs = []
        provs_labels = []
        for provision in national_implementation["provisions"]:
            prov_label = "__".join([f"{label['label'].lower()}_{label['value']}" for label in provision["link"]["itemsBase"] if label['value'] != ''])
            provs_labels.append(prov_label)

            provs.append(remove_html(provision["textEn"]))

        nested_texts.append(remove_html(transposition["text"]))
        data["nat_text"] += [provs]
        data["nat_id"] += [provs_labels]
        data["eu_id"] += [transposition["id"]]
        data["eu_text"] += [remove_html(transposition["text"])]

[['1. A European Investigation Order (EIO) is a judicial decision which has been issued or validated by a judicial authority of a Member State(‘the issuing State’) to have one or several specific investigative measure(s) carried out in another Member State (‘the executing State’) to obtain evidence in accordance with this Directive.', 'The EIO may also be issued for obtaining evidence that is already in the possession of the competent authorities of the executing State.', '2. Member States shall execute an EIO on the basis of the principle of mutual recognition and in accordance with this Directive.', '3. The issuing of an EIO may be requested by a suspected or accused person, or by a lawyer on his behalf, within the framework of applicable defence rights in conformity with national criminal procedure.', '4. This Directive shall not have the effect of modifying the obligation to respect the fundamental rights and legal principles as enshrined in Article 6 of the TEU, including the righ

In [8]:
t

[['1. A European Investigation Order (EIO) is a judicial decision which has been issued or validated by a judicial authority of a Member State(‘the issuing State’) to have one or several specific investigative measure(s) carried out in another Member State (‘the executing State’) to obtain evidence in accordance with this Directive.',
  'The EIO may also be issued for obtaining evidence that is already in the possession of the competent authorities of the executing State.',
  '2. Member States shall execute an EIO on the basis of the principle of mutual recognition and in accordance with this Directive.',
  '3. The issuing of an EIO may be requested by a suspected or accused person, or by a lawyer on his behalf, within the framework of applicable defence rights in conformity with national criminal procedure.',
  '4. This Directive shall not have the effect of modifying the obligation to respect the fundamental rights and legal principles as enshrined in Article 6 of the TEU, including 

In [9]:
import pandas as pd

d = pd.DataFrame(data)
d.shape

(4, 4)

In [10]:
d.head(25)

,eu_id,eu_text,nat_id,nat_text
0,art_1__par_3,3. The issuing of an EIO may be requested by a...,[article_5__alinea_2],[(2) The issuing of a European Investigative O...
1,art_2__let_c,(c) ‘issuing authority’ means:,[article_5__alinea_1],[Article 5. (1) A European Investigative Order...
2,art_2__let_d,(d) ‘executing authority’ means an authority ...,[article_9__alinea_1-2],[Article 9. (1) The following authorities in t...
3,art_4,An EIO may be issued:,[article_2__alinea_2],[Article 2. (1) ...(2) A European Investigativ...


In [11]:
d.to_pickle("transpositions.pickle")

In [12]:
transpositions["items"][21]

{'id': 'art_5__parMD5_3b6375f429c742185ff9996790ecd376',
 'fullId': 'chap_1__art_5__parMD5_3b6375f429c742185ff9996790ecd376',
 'type': 12,
 'text': '<p class="d-EURLEX d-inline d-p">The EIO shall, in particular, contain the following information:</p>',
 'heading': '',
 'subParagraph': '',
 'subHeading': '',
 'comment': None,
 'hasRecitals': False,
 'treeLevel': 2,
 'recitals': [],
 'articles': [],
 'oldArticles': [],
 'hasTransposition': None,
 'transp': None,
 'offence': None,
 'nationalActs': [],
 'regulationCompliance': None,
 'canTranspose': None}

In [13]:
transpositions["items"][22]

{'id': 'art_5__parMD5_3b6375f429c742185ff9996790ecd376__let_a',
 'fullId': 'chap_1__art_5__parMD5_3b6375f429c742185ff9996790ecd376__let_a',
 'type': 10,
 'text': '<p class="d-EURLEX d-inline d-p"><span class="d-EURLEX d-inline d-span">(a)  data about the issuing authority and, where applicable, the validating authority;</span></p>',
 'heading': '',
 'subParagraph': '',
 'subHeading': '',
 'comment': None,
 'hasRecitals': False,
 'treeLevel': 3,
 'recitals': [],
 'articles': [],
 'oldArticles': [],
 'hasTransposition': None,
 'transp': None,
 'offence': None,
 'nationalActs': [],
 'regulationCompliance': None,
 'canTranspose': None}

In [14]:
for r in d.iloc:
    print(r["eu_id"])
    print(r["eu_text"])
    print(r["nat_id"])
    print(r["nat_text"])
    print("===============")
    print("===============")

art_1__par_3
3. The issuing of an EIO may be requested by a suspected or accused person, or by a lawyer on his behalf, within the framework of applicable defence rights in conformity with national criminal procedure.
['article_5__alinea_2']
['(2) The issuing of a European Investigative Order may alternatively be requested by an accused person, a defendant or by a defence counsel authorised thereby for the implementation of the requisite defence in a criminal proceeding in accordance with the Criminal Procedure Code.']
art_2__let_c
(c)  ‘issuing authority’ means:
['article_5__alinea_1']
['Article 5. (1) A European Investigative Order in the Republic of Bulgaria shall be issued by:1. the relevant prosecutor: in the pre-trial proceeding;2. the relevant court: in the trial proceeding.']
art_2__let_d
(d)  ‘executing authority’ means an authority having competence to recognise an EIO and ensure its execution in accordance with this Directive and the procedures applicable in a similar domesti

In [15]:
# # for art_key, art_value in articles.items():
# d = {"eu_id": [], "eu_text": [], "nat_id": [], "nat_text": []}

# for art_key, art_value in articles.items():
#     art_att = {"eu_id": [], "eu_text": [], "nat_id": [], "nat_text": []}

#     if art_key != "art_1":
#         continue

#     for idx, art in enumerate(art_value):
#         if not art["hasTransposition"]:
#             aux_idx = idx
#             while aux_idx >= 0 and not art_value[aux_idx]["hasTransposition"]:
#                 aux_idx -= 1

#             if len(art_att["eu_id"]) > 0 and aux_idx >= 0:
#                 art_att["eu_id"][-1] = [art_att["eu_id"][-1]] + [art_value[idx]["id"]]
#                 art_att["eu_text"][-1] = [art_att["eu_text"][-1]] + [remove_html(art_value[idx]["text"])]
#             else:
#                 art_att["eu_id"].append((remove_long_label_element(art["id"])))
#                 art_att["eu_text"].append(remove_html(art["text"]))
#                 art_att["nat_id"].append("None")
#                 art_att["nat_text"].append("None")
#             continue

#         art_att["eu_id"] += [art["id"]]
#         art_att["eu_text"] += [remove_html(art["text"])]

#         for national_implementation in art["nationalActs"]:
#             provs = []
#             provs_labels = []
#             for provision in national_implementation["provisions"]:
#                 prov_label = "__".join([f"{label['label'].lower()}_{label['value']}" for label in provision["link"]["itemsBase"] if label['value'] != ''])
#                 provs_labels.append(prov_label)

#                 provs.append(remove_html(provision["textEn"]))
        
#             art_att["nat_text"] += [provs]
#             art_att["nat_id"] += [provs_labels]

#     for key in d.keys():
#         d[key] += art_att[key]

#     print(d)